In [1]:
import os
import re
import time
import gzip
import shutil
import requests
import numpy as np
import pandas as pd
from datetime import date

In [2]:
BASE_URL = "https://datasets.imdbws.com/"

FILES_IMDB = {
    "tit_bas": "title.basics.tsv",
    "tit_rate": "title.ratings.tsv",
    "name_bas": "name.basics.tsv",
    "tit_prin": "title.principals.tsv",
    "cast_crew": "title.crew.tsv",
}

FILES_HAND = {
    "add_seen": "add_movies_seen.txt",
    "add_unseen": "add_movies_unseen.txt",
    "add_secop": "add_movies_second_opinion.txt",
    "raw_status": "raw_status.xlsx"
}

FILES_GENERATED = {
    "films_raw": "films_raw.pkl",
    "films_reading": "films_reading.xlsx"
}

In [3]:
# 1 load and clean the watched movies data
ids_and_status = os.path.join("data", "handcrafted", FILES_HAND["raw_status"])
watched = pd.read_excel(ids_and_status)
watched["tconst"] = watched["tconst"].str.strip()

# 2 load the files that are needed to extend the watched movies
title_basics_path = os.path.join("data", "imdb", FILES_IMDB["tit_bas"])
title_basics = pd.read_csv(title_basics_path, sep="\t", low_memory=False)
name_basics_path = os.path.join("data", "imdb", FILES_IMDB["name_bas"])
name_basics = pd.read_csv(name_basics_path, sep="\t", low_memory=False)
title_rate_path = os.path.join("data", "imdb", FILES_IMDB["tit_rate"])
title_rate = pd.read_csv(title_rate_path, sep="\t")
title_crew_path = os.path.join("data", "imdb", FILES_IMDB["cast_crew"])
title_crew = pd.read_csv(title_crew_path, sep="\t", low_memory=False)
title_prin_mega_path = os.path.join("data", "imdb", FILES_IMDB["tit_prin"])

In [4]:
# watched = addCastAndCrew(watched, title_prin_mega_path)
watched_tconst = watched.loc[:,"tconst"].to_frame()
watched_tconst

,tconst
0,tt0015324
1,tt0017136
2,tt0022100
3,tt0025316
4,tt0031381
...,...
612,tt9691136
613,tt9731598
614,tt9764362
615,tt9783600


In [5]:
# watched_title_rate = watched_tconst
# title_prin_mega_path = title_prin_mega_path

# get cast and crew from the big file
watched_films_cast  = pd.DataFrame(columns=["tconst", "ordering","nconst", "category", "job", "characters"])
counter = 1
for chunk in pd.read_csv(title_prin_mega_path, sep="\t", chunksize=100000):
    rows = pd.merge(watched_tconst.loc[:,"tconst"], chunk, on="tconst", how="inner")
    watched_films_cast = pd.concat([rows,watched_films_cast], ignore_index = True)
    print("chunk #",counter)
    counter += 1
watched_films_cast.replace(to_replace = "\\N", value = np.nan, inplace=True)
watched_films_cast.drop(['characters'], axis=1, inplace=True)

chunk # 1
chunk # 2
chunk # 3
chunk # 4
chunk # 5
chunk # 6
chunk # 7
chunk # 8
chunk # 9
chunk # 10
chunk # 11
chunk # 12
chunk # 13
chunk # 14
chunk # 15
chunk # 16
chunk # 17
chunk # 18
chunk # 19
chunk # 20
chunk # 21
chunk # 22
chunk # 23
chunk # 24
chunk # 25
chunk # 26
chunk # 27
chunk # 28
chunk # 29
chunk # 30
chunk # 31
chunk # 32
chunk # 33
chunk # 34
chunk # 35
chunk # 36
chunk # 37
chunk # 38
chunk # 39
chunk # 40
chunk # 41
chunk # 42
chunk # 43
chunk # 44
chunk # 45
chunk # 46
chunk # 47
chunk # 48
chunk # 49
chunk # 50
chunk # 51
chunk # 52
chunk # 53
chunk # 54
chunk # 55
chunk # 56
chunk # 57
chunk # 58
chunk # 59
chunk # 60
chunk # 61
chunk # 62
chunk # 63
chunk # 64
chunk # 65
chunk # 66
chunk # 67
chunk # 68
chunk # 69
chunk # 70
chunk # 71
chunk # 72
chunk # 73
chunk # 74
chunk # 75
chunk # 76
chunk # 77
chunk # 78
chunk # 79
chunk # 80
chunk # 81
chunk # 82
chunk # 83
chunk # 84
chunk # 85
chunk # 86
chunk # 87
chunk # 88
chunk # 89
chunk # 90
chunk # 91
chunk # 

In [6]:
watched_films_cast

,tconst,ordering,nconst,category,job
0,tt9806192,10,nm10909656,production_designer,NaN
1,tt9806192,1,nm7079932,actor,NaN
2,tt9806192,2,nm4973460,actress,NaN
3,tt9806192,3,nm1253936,actor,NaN
4,tt9806192,4,nm11127862,actor,NaN
...,...,...,...,...,...
6115,tt0025316,5,nm0001008,director,NaN
6116,tt0025316,6,nm0728307,writer,screen play
6117,tt0025316,7,nm0011343,writer,based on the short story by
6118,tt0025316,8,nm0907900,cinematographer,NaN


In [7]:
# split fetched cast and crew on values [director, writer] and other
direct = watched_films_cast.category == 'director'
writer = watched_films_cast.category == 'writer'
both = direct | writer
# this isn't needed yet, takes a lot of space
principals_writer_direct = watched_films_cast.loc[both,:]
principals_other = watched_films_cast.loc[~both,:]
principals_writer_direct

,tconst,ordering,nconst,category,job
5,tt9806192,5,nm3021346,director,NaN
6,tt9806192,6,nm0491011,writer,adaptation and dialogue
15,tt9783600,5,nm2676052,director,NaN
16,tt9783600,6,nm2808422,writer,"based on the short story ""Escape from Spiderhe..."
17,tt9783600,7,nm1014201,writer,screenplay by
...,...,...,...,...,...
6106,tt0022100,6,nm0902376,writer,script
6107,tt0022100,7,nm0414825,writer,article
6115,tt0025316,5,nm0001008,director,NaN
6116,tt0025316,6,nm0728307,writer,screen play


In [8]:
title_crew.replace(to_replace = "\\N", value = np.nan, inplace=True)

In [9]:
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN
...,...,...,...
9634763,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9634764,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9634765,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9634766,tt9916856,nm10538645,nm6951431


In [10]:
# get the directors and writers for our movie id's (tconst); reformat and add them together

directors = title_crew.loc[~title_crew.loc[:,'directors'].isna(),['tconst','directors']]
needed_directors = pd.merge(watched_tconst.loc[:,"tconst"], directors, on="tconst", how="left")
needed_directors.directors = needed_directors.directors.str.split(',')
needed_directors = needed_directors.explode('directors')
needed_directors.loc[:,'category'] = 'director'
needed_directors.rename(columns={"directors": "nconst"}, inplace=True)

writers = title_crew.loc[~title_crew.loc[:,'writers'].isna(),['tconst','writers']]
needed_writers = pd.merge(watched_tconst.loc[:,"tconst"], writers, on="tconst", how="left")
needed_writers.writers = needed_writers.writers.str.split(',')
needed_writers = needed_writers.explode('writers')
needed_writers.loc[:,'category'] = 'writer'
needed_writers.rename(columns={"writers": "nconst"}, inplace=True)

needed_directors_writers = pd.concat([needed_directors, needed_writers], ignore_index=True)

In [11]:
needed_directors_writers

,tconst,nconst,category
0,tt0015324,nm0000036,director
1,tt0017136,nm0000485,director
2,tt0022100,nm0000485,director
3,tt0025316,nm0001008,director
4,tt0031381,nm0281808,director
...,...,...,...
2545,tt9783600,nm2808422,writer
2546,tt9783600,nm1014201,writer
2547,tt9783600,nm1116660,writer
2548,tt9806192,nm3021346,writer


now take needed_directors_writers and fill it with the ordering and job from principals_writer_direct.

after that append principals_other to it.

than we can add name basics to it.

then this table must be exploded on every possible way (check for lists and comma's in strings)

and then we can finally add it to the final thing.

In [21]:
# now take needed_directors_writers and fill it with the ordering and job from principals_writer_direct.
needed_dir_writ_principals = pd.merge(needed_directors_writers, principals_writer_direct, on=["tconst",'nconst'], how="left")
needed_dir_writ_principals = needed_dir_writ_principals.drop('category_y', axis=1)
needed_dir_writ_principals.rename(columns={'category_x': "category"}, inplace=True)

# add the non-director / writer principials
principals_better = pd.concat([principals_other, needed_dir_writ_principals],ignore_index=True)

# add name_basics
principals_better = pd.merge(principals_better, name_basics, on=['nconst'], how="left")

In [22]:
# clean data now it still isn't that much
principals_better.ordering = principals_better.ordering.astype("Int64")
principals_better = principals_better.replace(to_replace = "\\N", value = np.nan)
principals_better.birthYear = principals_better.birthYear.astype("Int64")
principals_better.deathYear = principals_better.deathYear.astype("Int64")
principals_better.primaryProfession = principals_better.primaryProfession.str.split(',')
principals_better.knownForTitles = principals_better.knownForTitles.str.split(',')

In [23]:
# explode the data
principals_better = principals_better.explode('primaryProfession')
principals_better = principals_better.explode('knownForTitles')
principals_better

,tconst,ordering,nconst,category,job,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt9806192,10,nm10909656,production_designer,NaN,Jocelyn Charles,<NA>,<NA>,director,tt13074948
0,tt9806192,10,nm10909656,production_designer,NaN,Jocelyn Charles,<NA>,<NA>,director,tt13210250
0,tt9806192,10,nm10909656,production_designer,NaN,Jocelyn Charles,<NA>,<NA>,director,tt9806192
0,tt9806192,10,nm10909656,production_designer,NaN,Jocelyn Charles,<NA>,<NA>,writer,tt13074948
0,tt9806192,10,nm10909656,production_designer,NaN,Jocelyn Charles,<NA>,<NA>,writer,tt13210250
...,...,...,...,...,...,...,...,...,...,...
6826,tt9806192,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,writer,tt0344510
6826,tt9806192,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt1149361
6826,tt9806192,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt0211915
6826,tt9806192,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt9806192


In [27]:
watched = pd.merge(watched, principals_better, on=['tconst'], how="left")
watched

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,ordering,nconst,category,job,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0015324,0,NaT,NaN,NaN,NaN,NaN,10,nm0504380,cinematographer,NaN,Elgin Lessley,1883,1944,cinematographer,tt0016332
1,tt0015324,0,NaT,NaN,NaN,NaN,NaN,10,nm0504380,cinematographer,NaN,Elgin Lessley,1883,1944,cinematographer,tt0015163
2,tt0015324,0,NaT,NaN,NaN,NaN,NaN,10,nm0504380,cinematographer,NaN,Elgin Lessley,1883,1944,cinematographer,tt0014341
3,tt0015324,0,NaT,NaN,NaN,NaN,NaN,10,nm0504380,cinematographer,NaN,Elgin Lessley,1883,1944,cinematographer,tt0015324
4,tt0015324,0,NaT,NaN,NaN,NaN,NaN,10,nm0504380,cinematographer,NaN,Elgin Lessley,1883,1944,actor,tt0016332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71987,tt9806192,1,NaT,NaN,NaN,4.0,NaN,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,writer,tt0344510
71988,tt9806192,1,NaT,NaN,NaN,4.0,NaN,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt1149361
71989,tt9806192,1,NaT,NaN,NaN,4.0,NaN,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt0211915
71990,tt9806192,1,NaT,NaN,NaN,4.0,NaN,6,nm0491011,writer,adaptation and dialogue,Guillaume Laurant,1961,<NA>,actor,tt9806192
